In [172]:
import pandas as pd
import datetime
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Model prediction pipeling using within 1 km instead of 15 km

In [173]:
minedex = pd.read_csv('recent_mindex.csv')
minedex.shape

(1222, 13)

In [174]:
# Acess Site.csv for lat and lon.
site_locs = pd.read_excel('Sites.xlsx')
site_locs = site_locs[['SiteCode', 'Latitude', 'Longitude']].set_index('SiteCode')
site_locs = site_locs.rename(columns={
    "Latitude": "lat",
    "Longitude": "lon"
})

In [175]:
# minedex = minedex.merge(site_locs, left_on="SiteCode", right_index=True)

In [176]:
full_to_atomic = {'Hydrogen' : 'H', 'Helium' : 'He', 'Lithium' : 'Li', 'Beryllium' : 'Be', 'Boron' : 'B', 'Carbon' : 'C', 'Nitrogen' : 'N', 'Oxygen' : 'O', 'Fluorine' : 'F', 'Neon' : 'Ne', 'Sodium' : 'Na', 'Magnesium' : 'Mg', 'Aluminium' : 'Al', 'Silicon' : 'Si', 'Phosphorus' : 'P', 'Sulfur' : 'S', 'Chlorine' : 'Cl', 'Argon' : 'Ar', 'Potassium' : 'K', 'Calcium' : 'Ca', 'Scandium' : 'Sc', 'Titanium' : 'Ti', 'Vanadium' : 'V', 'Chromium' : 'Cr', 'Manganese' : 'Mn', 'Iron' : 'Fe', 'Cobalt' : 'Co', 'Nickel' : 'Ni', 'Copper' : 'Cu', 'Zinc' : 'Zn', 'Gallium' : 'Ga', 'Germanium' : 'Ge', 'Arsenic' : 'As', 'Selenium' : 'Se', 'Bromine' : 'Br', 'Krypton' : 'Kr', 'Rubidium' : 'Rb', 'Strontium' : 'Sr', 'Yttrium' : 'Y', 'Zirconium' : 'Zr', 'Niobium' : 'Nb', 'Molybdenum' : 'Mo', 'Technetium' : 'Tc', 'Ruthenium' : 'Ru', 'Rhodium' : 'Rh', 'Palladium' : 'Pd', 'Silver' : 'Ag', 'Cadmium' : 'Cd', 'Indium' : 'In', 'Tin' : 'Sn', 'Antimony' : 'Sb', 'Tellurium' : 'Te', 'Iodine' : 'I', 'Xenon' : 'Xe', 'Cesium' : 'Cs', 'Barium' : 'Ba', 'Lanthanum' : 'La', 'Cerium' : 'Ce', 'Praseodymium' : 'Pr', 'Neodymium' : 'Nd', 'Promethium' : 'Pm', 'Samarium' : 'Sm', 'Europium' : 'Eu', 'Gadolinium' : 'Gd', 'Terbium' : 'Tb', 'Dysprosium' : 'Dy', 'Holmium' : 'Ho', 'Erbium' : 'Er', 'Thulium' : 'Tm', 'Ytterbium' : 'Yb', 'Lutetium' : 'Lu', 'Hafnium' : 'Hf', 'Tantalum' : 'Ta', 'Tungsten' : 'W', 'Rhenium' : 'Re', 'Osmium' : 'Os', 'Iridium' : 'Ir', 'Platinum' : 'Pt', 'Gold' : 'Au', 'Mercury' : 'Hg', 'Thallium' : 'Tl', 'Lead' : 'Pb', 'Bismuth' : 'Bi', 'Polonium' : 'Po', 'Astatine' : 'At', 'Radon' : 'Rn', 'Francium' : 'Fr', 'Radium' : 'Ra', 'Actinium' : 'Ac', 'Thorium' : 'Th', 'Protactinium' : 'Pa', 'Uranium' : 'U', 'Neptunium' : 'Np', 'Plutonium' : 'Pu', 'Americium' : 'Am', 'Curium' : 'Cm', 'Berkelium' : 'Bk', 'Californium' : 'Cf', 'Einsteinium' : 'Es', 'Fermium' : 'Fm', 'Mendelevium' : 'Md', 'Nobelium' : 'No', 'Lawrencium' : 'Lr', 'Rutherfordium' : 'Rf', 'Dubnium' : 'Db', 'Seaborgium' : 'Sg', 'Bohrium' : 'Bh', 'Hassium' : 'Hs', 'Meitnerium' : 'Mt', 'Darmstadtium' : 'Ds', 'Roentgenium' : 'Rg', 'Copernicium' : 'Cn', 'Nihonium' : 'Nh', 'Flerovium' : 'Fl', 'Moscovium' : 'Mc', 'Livermorium' : 'Lv', 'Tennessine' : 'Ts', 'Og' : 'Oganesson',}

def convert_to_atomic(col):
    ### NEED TO CATER TO EDGE CASES
    ### e.g Mag Oxide, Alumina etc.
    ###
    full_name = col
    
    if full_name in full_to_atomic.keys():
        atomic = full_to_atomic[full_name]
        return atomic

    else:
        return np.nan
        

minedex['CommodityName'] = minedex['CommodityName'].apply(convert_to_atomic)

In [177]:
### DROPPING WIERD COMMODS FOR NOW.
minedex = minedex[~minedex['CommodityName'].isna()]

In [178]:
# get list of unique mineral combinations
list_combinations = minedex.groupby('SiteCode')['CommodityName'].unique().to_list()

# sort alphabetically
list_combinations = [sorted(commods) for commods in list_combinations]

unique_commods = []
for commods in list_combinations:
    if commods not in unique_commods:
        unique_commods.append(commods)

In [179]:
# WAMEX CLEANING
def replace_neg9999(val):
    if val == -9999: 
        return 0
    else:
        return val
    
    
def abs_halve_neg_val(val):
    if val < 0: 
        return abs(val) / 2
    else: 
        return val

def clean_assays(df):
    df = df.applymap(replace_neg9999)
    df = df.applymap(abs_halve_neg_val)

    # all 9999 -> 0 values are removed
    indexs_0 = df[df.values == 0].index
    df = df.drop(index=indexs_0)
    
    return df

def abundant_ratio(row, abundant_mineral):
    return row / row[abundant_mineral]

def make_pickle_fn(df, model, prefix='../../pickles/'):
    fn = f'{prefix}{type(model).__name__}-'
    for i in df.columns:
        fn = fn + i + '-'

    fn = fn[:-1] + '.pickle'   
    return fn

def filter_by_wamex_minerals(unique_commods, wamex_unique_minerals):
    # filter unique commods by availability in wamex assays
    filtered_unique_commods = []
    for commods in unique_commods:
        filtered_commods = []
        for commod in commods:
            if commod in wamex_unique_minerals:
                filtered_commods.append(commod)
        filtered_unique_commods.append(filtered_commods)
    return filtered_unique_commods

In [180]:
# WAMEX
wamex = pd.read_csv('Merged_Ni_lessthan15km.csv',index_col=0)
# wamex.head()

## Filter by < 1 km. 

In [181]:
#need to change distance column to numeric
def dist_to_float(val):
        val = val[0:-3]
        return float(val)

In [182]:
for i in range(0, len(wamex)):
    wamex.at[i, 'distance'] = dist_to_float(wamex.at[i, 'distance'])

In [183]:
wamex = wamex[wamex['distance']<=1]
wamex.shape #there are still more than 5000 relevant rows

(5835, 19)

In [184]:
wamex = wamex.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'lat', 'long', 
                     'minedex_coords', 'minedex_site',
                    'distance'])

wamex = wamex.rename(columns={
    "wamex_lat": "lat",
    "wamex_lon": "lon"
})

wamex = wamex.drop_duplicates()

In [185]:
wamex_minerals = wamex.drop(columns=['lat', 'lon'])

wamex.loc[:, wamex_minerals.columns] = clean_assays(wamex.loc[:,wamex_minerals.columns])

# Ratio transform
wamex.loc[:, wamex_minerals.columns] = wamex.loc[:, wamex_minerals.columns].apply(
                                                            abundant_ratio, abundant_mineral='Ni',
                                                            axis=1)

# Available minerals in WAMEX
wamex_unique_minerals = wamex_minerals.columns
filtered_unique_commods = filter_by_wamex_minerals(unique_commods, wamex_unique_minerals)

In [186]:
all_unique_commods = []
for cl in filtered_unique_commods:
    all_unique_commods.extend(cl)

In [187]:
wamex_commods = list(set(all_unique_commods))
wamex_commods

['Au', 'Fe', 'Ag', 'Cu', 'Co', 'Rh', 'S', 'Pd', 'Pt', 'Ni']

In [188]:
minedex = minedex[['SiteCode', 'Date', 'CommodityName', 'ppm']]

# sum up duplicate ppm values 
minedex['ppm'] = minedex.groupby(['SiteCode', 'CommodityName'])['ppm'].transform('sum')

# drop duplicate ppm values.
minedex = minedex.drop_duplicates()

In [189]:
# Filter by WAMEX available commods
minedex = minedex[minedex['CommodityName'].isin(wamex_commods)]

In [190]:
minedex_piv = minedex[['SiteCode', 'CommodityName', 'ppm']]

In [191]:
unique_idxs = minedex_piv[['SiteCode', 'CommodityName']].drop_duplicates().index

# don't know why these are here.
minedex_piv[~minedex_piv.index.isin(unique_idxs)]
minedex_piv = minedex_piv[minedex_piv.index.isin(unique_idxs)]

### Prediction Set

In [192]:
# create a pivot table by SiteCode
minedex_piv = minedex_piv.pivot(index='SiteCode', columns='CommodityName', values='ppm').rename_axis(None)

In [193]:
minedex_piv.columns = minedex_piv.columns.rename(None)

In [194]:
# workaround to get merge to work.
minedex_piv = minedex_piv.reset_index().merge(site_locs, left_on='index', right_index=True)
minedex_piv

,index,Ag,Au,Co,Cu,Fe,Ni,Pd,Pt,Rh,S,lat,lon
0,S0000192,NaN,NaN,NaN,NaN,NaN,59000.000000,NaN,NaN,NaN,NaN,-32.416982,119.689058
1,S0000539,NaN,NaN,NaN,NaN,NaN,20000.000000,NaN,NaN,NaN,NaN,-31.011440,121.680000
2,S0001174,NaN,NaN,500.0,53.640000,237400.000000,15000.000000,NaN,NaN,NaN,NaN,-33.633382,120.374303
3,S0001178,NaN,NaN,NaN,NaN,NaN,18500.000000,NaN,NaN,NaN,NaN,-30.393502,121.648875
4,S0001188,NaN,NaN,NaN,NaN,NaN,16500.000000,NaN,NaN,NaN,NaN,-30.736222,121.802770
5,S0001197,NaN,NaN,NaN,NaN,NaN,14000.000000,NaN,NaN,NaN,NaN,-26.037849,116.387001
5,S0001197,NaN,NaN,NaN,NaN,NaN,14000.000000,NaN,NaN,NaN,NaN,-26.037849,116.387001
6,S0001204,NaN,NaN,NaN,22000.000000,NaN,72000.000000,NaN,NaN,NaN,NaN,-30.065901,121.626099
7,S0001218,NaN,NaN,NaN,NaN,NaN,28000.000000,NaN,NaN,NaN,NaN,-32.582031,119.741699
8,S0001221,NaN,NaN,NaN,NaN,NaN,48000.000000,NaN,NaN,NaN,NaN,-31.457821,121.539299


In [195]:
# for each site, make a string of non-null minerals. 
# template: LinearRegression-Ag-Au-Co-Cu-Pd-Pt-lat-lon

minedex_minerals = minedex_piv.drop(columns=['index', 'lat', 'lon'])

def commod_string(row):
    commod_str = "-".join(row.dropna().index.to_list()).replace('-Co', "").replace('Co', "")
    if commod_str.startswith('-'):
        commod_str = commod_str[1:]
    
    row['commod_str'] = commod_str
    return row

minedex_minerals_str = minedex_minerals.apply(commod_string, axis=1)

In [196]:
"Ni-Cu".replace('Co', "")

'Ni-Cu'

In [198]:
minedex_minerals_str['SiteCode'] = minedex_piv['index']
minedex_minerals_str

,Ag,Au,Co,Cu,Fe,Ni,Pd,Pt,Rh,S,commod_str,SiteCode
0,NaN,NaN,NaN,NaN,NaN,59000.000000,NaN,NaN,NaN,NaN,Ni,S0000192
1,NaN,NaN,NaN,NaN,NaN,20000.000000,NaN,NaN,NaN,NaN,Ni,S0000539
2,NaN,NaN,500.0,53.640000,237400.000000,15000.000000,NaN,NaN,NaN,NaN,Cu-Fe-Ni,S0001174
3,NaN,NaN,NaN,NaN,NaN,18500.000000,NaN,NaN,NaN,NaN,Ni,S0001178
4,NaN,NaN,NaN,NaN,NaN,16500.000000,NaN,NaN,NaN,NaN,Ni,S0001188
5,NaN,NaN,NaN,NaN,NaN,14000.000000,NaN,NaN,NaN,NaN,Ni,S0001197
5,NaN,NaN,NaN,NaN,NaN,14000.000000,NaN,NaN,NaN,NaN,Ni,S0001197
6,NaN,NaN,NaN,22000.000000,NaN,72000.000000,NaN,NaN,NaN,NaN,Cu-Ni,S0001204
7,NaN,NaN,NaN,NaN,NaN,28000.000000,NaN,NaN,NaN,NaN,Ni,S0001218
8,NaN,NaN,NaN,NaN,NaN,48000.000000,NaN,NaN,NaN,NaN,Ni,S0001221


In [199]:
# Add lat and lon
minedex_min_loc = minedex_minerals_str.merge(site_locs, left_on="SiteCode", right_index=True)

In [200]:
minerals_df = minedex_min_loc.drop(columns=['lat', 'lon', 'commod_str', 'SiteCode'])

orig_ni = minerals_df['Ni']

#Ratio transform
# minedex_min_loc.loc[:, minerals_df.columns] = minedex_min_loc.loc[:, minerals_df.columns].apply(
#                                                             abundant_ratio, abundant_mineral='Ni', axis=1)
# apply Log transform
minedex_min_loc.loc[:, minerals_df.columns] = minedex_min_loc.loc[:,minerals_df.columns].apply(np.log, axis=1)

In [201]:
# keep ref of sitecode to index
site_codes_ref = minedex_min_loc['SiteCode']

In [202]:
def load_commod_model(commod_str, model='LinearRegression'):
    with open(f'pickles/{model}-{commod_str}-lat-lon.pickle', 'rb') as file:
        m = pickle.load(file)    
    return m

In [203]:
# create a list of dataframes by unique commodity groups.
gb = minedex_min_loc.groupby('commod_str')    
list_of_commod_dfs = [gb.get_group(x) for x in gb.groups]

In [204]:
def create_site_pred(commod_group_df):
    '''Gets a dataframe containing a unique commodity group,
    '''
    # drop empty mineral cols
    commod_group_df = commod_group_df.dropna(axis=1)
    # dont know why duplicates
    commod_group_df = commod_group_df.drop_duplicates()
    # create X vars from data.
#     commod_group_df_X = commod_group_df.drop(columns=['commod_str', 'SiteCode', 'Ni'])
    commod_group_df_X = commod_group_df.drop(columns=['commod_str', 'SiteCode'])
    # if site already has a Cobalt value, drop.
    if 'Co' in commod_group_df_X.columns:
        commod_group_df_X = commod_group_df_X.drop(columns=['Co'])
    
    commod_str = commod_group_df['commod_str'].iloc[0]
    
    try:
        # use first commod string to find relevant model pickle.
        # Linear Regression
        lr = load_commod_model(commod_str=commod_str, model='LinearRegression')
        lr_preds = lr.predict(commod_group_df_X)
        commod_group_df['Co_pred_lr'] = lr_preds
        # Random Forest
        rf = load_commod_model(commod_str=commod_str, model='RandomForestRegressor')
        rf_preds = rf.predict(commod_group_df_X)
        commod_group_df['Co_pred_rf'] = rf_preds
        
        # make commod_str column
        commod_group_df['commods_used'] = commod_str
            
        site_pred_df = commod_group_df[['SiteCode', 'Co_pred_lr', 'Co_pred_rf', 'commods_used']]

        return site_pred_df
    
    except:
        print(f'Could not find models for {commod_str}')

In [205]:
list_of_site_pred_dfs = []

for commod_group_df in list_of_commod_dfs:
    site_pred_df = create_site_pred(commod_group_df)
    list_of_site_pred_dfs.append(site_pred_df)

In [206]:
all_site_preds_df = pd.concat(list_of_site_pred_dfs, axis=0)

In [207]:
# frik knows why the merge creates duplicates!!!!
all_site_preds_df = all_site_preds_df.merge(minedex[['SiteCode', 'Date']], on='SiteCode').drop_duplicates()

In [284]:
# get original Ni values for inveser transform
# all_site_preds_df = all_site_preds_df.merge(orig_ni, left_index=True, right_index=True).drop_duplicates()

### Invert log

In [208]:
all_site_preds_df['Co_pred_lr'] = np.exp(all_site_preds_df['Co_pred_lr'])  #* all_site_preds_df['Ni'] 
all_site_preds_df['Co_pred_rf'] = np.exp(all_site_preds_df['Co_pred_rf'])  #* all_site_preds_df['Ni'] 

In [209]:
all_site_preds_df

,SiteCode,Co_pred_lr,Co_pred_rf,commods_used,Date
0,S0230165,325.966213,136.200878,Ag-Au-Cu-Ni,22/06/2020
4,S0018904,106.187394,199.545264,Ag-Au-Cu-Ni-Pd-Pt,09/12/2020
11,S0019102,109.811089,205.148395,Ag-Au-Cu-Ni-Pd-Pt,09/12/2020
18,S0001805,506.485016,371.500041,Au-Cu-Ni-Pd-Pt,30/06/2015
23,S0004096,78.037777,87.374002,Au-Cu-Ni-Pd-Pt,30/11/2020
28,S0032888,87.714722,88.739152,Au-Cu-Ni-Pd-Pt,04/03/2019
33,S0001806,0.006172,276.085597,Au-Cu-Ni-Pd-Pt-Rh,26/09/2008
39,S0230680,0.006172,276.085597,Au-Cu-Ni-Pd-Pt-Rh,26/09/2008
45,S0001174,510.740074,239.226274,Cu-Fe-Ni,31/12/2017
49,S0005631,449.434225,230.990750,Cu-Fe-Ni,31/12/2017


In [210]:
# all_site_preds_df.merge(true_cobalt)

In [211]:
pd.set_option('display.max_rows', 500)

# add cobalt on
true_cobalt = minedex[minedex['CommodityName'] == 'Co'][['SiteCode', 'ppm']]
all_site_preds_df = all_site_preds_df.merge(true_cobalt, on='SiteCode')
all_site_preds_df = all_site_preds_df.rename(columns={'ppm': 'Co_true'})
# all_site_preds_df

In [212]:
all_site_preds_df = all_site_preds_df[['SiteCode', 'Date', 'commods_used', 'Co_pred_lr', 'Co_pred_rf', 'Co_true']]

In [213]:
all_site_preds_df.to_csv('preds_lr_rf.csv')

In [214]:
all_site_preds_df.commods_used.nunique()

10

In [215]:
plot_preds_df = all_site_preds_df[['Co_pred_lr', 'Co_pred_rf', 'Co_true', 'commods_used']].set_index('commods_used')

In [216]:
plot_preds_df

,Co_pred_lr,Co_pred_rf,Co_true
commods_used,,,
Ag-Au-Cu-Ni-Pd-Pt,106.187394,199.545264,410.0
Ag-Au-Cu-Ni-Pd-Pt,109.811089,205.148395,360.0
Cu-Fe-Ni,510.740074,239.226274,500.0
Cu-Fe-Ni,449.434225,230.990750,600.0
Cu-Fe-Ni,264.768381,163.675136,200.0
Cu-Fe-Ni,546.397447,227.452511,800.0
Cu-Fe-Ni,259.127655,167.172137,200.0
Cu-Fe-Ni-S,407.373268,165.462728,500.0
Cu-Fe-Ni-S,96.679400,143.756634,130.0


In [217]:
len(plot_preds_df)

129

In [223]:
err_df = pd.read_csv('data/model_errors_1km.csv', index_col=0)

In [224]:
def clean_lr_path(row):
    row = row.split('LinearRegression')[1].replace('.pickle','')
    if row.startswith('-'):
        row = row[1:]        
    return row.replace('-lat-lon', '')

err_df['lr_path'] = err_df['lr_path'].apply(clean_lr_path)

In [225]:
err_df.head()

,lr_path,lr_train_mse,lr_test_mse,rf_train_mse,rf_test_mse
0,Ni,0.965767,1.068373,0.056764,0.415347
1,Ni,0.965767,1.068373,0.055750,0.423818
2,Cu-Fe-Ni,0.895495,0.821539,0.056776,0.424473
3,Cu-Ni,0.947094,0.848596,0.058551,0.323242
4,Cu-Ni-S,0.835847,0.746430,0.063178,0.386450


In [226]:
len(err_df)

27

In [227]:
all_site_preds_df = all_site_preds_df.merge(err_df.set_index('lr_path'), left_on='commods_used', right_index=True).drop_duplicates(subset='SiteCode')
all_site_preds_df

,SiteCode,Date,commods_used,Co_pred_lr,Co_pred_rf,Co_true,lr_train_mse,lr_test_mse,rf_train_mse,rf_test_mse
0,S0018904,09/12/2020,Ag-Au-Cu-Ni-Pd-Pt,106.187394,199.545264,410.0,0.518070,0.616021,0.075776,0.473738
1,S0019102,09/12/2020,Ag-Au-Cu-Ni-Pd-Pt,109.811089,205.148395,360.0,0.518070,0.616021,0.075776,0.473738
2,S0001174,31/12/2017,Cu-Fe-Ni,510.740074,239.226274,500.0,0.895495,0.821539,0.056776,0.424473
3,S0005631,31/12/2017,Cu-Fe-Ni,449.434225,230.990750,600.0,0.895495,0.821539,0.056776,0.424473
4,S0006586,31/12/2012,Cu-Fe-Ni,264.768381,163.675136,200.0,0.895495,0.821539,0.056776,0.424473
5,S0006685,31/12/2012,Cu-Fe-Ni,546.397447,227.452511,800.0,0.895495,0.821539,0.056776,0.424473
6,S0229459,31/12/2015,Cu-Fe-Ni,259.127655,167.172137,200.0,0.895495,0.821539,0.056776,0.424473
7,S0020966,13/11/2019,Cu-Fe-Ni-S,407.373268,165.462728,500.0,0.777399,0.655102,0.066049,0.379905
8,S0224398,24/02/2015,Cu-Fe-Ni-S,96.679400,143.756634,130.0,0.777399,0.655102,0.066049,0.379905
9,S0001442,02/02/2006,Cu-Ni,559.154081,519.956198,1000.0,0.947094,0.848596,0.058551,0.323242


In [228]:
all_site_preds_df.to_csv('all_site_preds_with_err_no_duplicates_1km.csv')

In [170]:
mean_squared_error(all_site_preds_df['Co_pred_lr'], all_site_preds_df['Co_true'])

842275.6253506433

In [171]:
mean_squared_error(all_site_preds_df['Co_pred_rf'], all_site_preds_df['Co_true'])

998927.3095540924